In [3]:
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType,DecimalType,DoubleType,BooleanType,TimestampType
from pyspark.shell import spark

from pyspark.sql.functions import initcap, lower,col,year,month,dayofmonth,expr
from pyspark.sql import functions as sf
from pyspark.sql.functions import regexp_replace
import datetime
import pandas as pd
import numpy as np
 

#import org.apache.spark.sql.functions.format_number




os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark_app = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()



Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.9.12 (main, Apr  4 2022 05:22:27)
Spark context Web UI available at http://LAPTOP-PSIMHJKQ:4041
Spark context available as 'sc' (master = local[*], app id = local-1661604817744).
SparkSession available as 'spark'.


# Functional Requirement 1.1
1.1 Functional Requirements - Load Credit Card Database -cdw_sapp_credit(SQL)


In [12]:

#cdw_sapp_f_creditcard details
credit_df = spark.read.format('org.apache.spark.sql.json').load('C:/Users/asha_/dev/capstoneproj/Data/cdw_sapp_credit.json')
#df_credit1.show(5)
#df_credit1.printSchema()

In [13]:
credit_df.createOrReplaceTempView("credit_table")

In [14]:
spark.sql("select * from credit_table").show(3)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
only showing top 3 rows



In [15]:
credit_df = spark.sql('SELECT CREDIT_CARD_NO CUST_CC_NO,\
         CONCAT(YEAR, LPAD(Month, 2, 0), \
         LPAD(Day, 2, 0)) TIMEID, \
         CUST_SSN,\
         BRANCH_CODE,\
         TRANSACTION_TYPE,\
         TRANSACTION_VALUE,\
         TRANSACTION_ID \
 FROM credit_table')

In [17]:
credit_df.show(5)
credit_df.printSchema()

+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653349028689|20180214|123459988|        114|       Education|             78.9|             1|
|4210653349028689|20180320|123459988|         35|   Entertainment|            14.24|             2|
|4210653349028689|20180708|123459988|        160|         Grocery|             56.7|             3|
|4210653349028689|20180419|123459988|        114|   Entertainment|            59.73|             4|
|4210653349028689|20181010|123459988|         93|             Gas|             3.59|             5|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
only showing top 5 rows

root
 |-- CUST_CC_NO: string (nullable = true)
 |-- TIMEID: string (nullabl

In [19]:
#changing datatype of columns

#credit_df = credit_df.withColumn("TIMEID",col("TIMEID").cast("String"))
credit_df = credit_df.withColumn("CUST_SSN",col("CUST_SSN").cast("int"))
credit_df = credit_df.withColumn("TRANSACTION_ID",col("TRANSACTION_ID").cast("int"))
credit_df = credit_df.withColumn("BRANCH_CODE",col("BRANCH_CODE").cast("int"))
#df_credit2.show(3)
credit_df.printSchema()

root
 |-- CUST_CC_NO: string (nullable = true)
 |-- TIMEID: string (nullable = true)
 |-- CUST_SSN: integer (nullable = true)
 |-- BRANCH_CODE: integer (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- TRANSACTION_ID: integer (nullable = true)



In [3]:
#created column named 'timed'
#df_credit2 = df_credit1.withColumn("TIMEID", expr("make_date(Year,Month,Day)"))




In [20]:
# column renamed from credit_card_no to cust_cc_no
#df_credit2 = df_credit2.withColumnRenamed("CREDIT_CARD_NO","CUST_CC_NO")

In [21]:

#drop columns day,month,year
#df_credit2 = df_credit2.drop('DAY','MONTH','YEAR')
#df_credit2.printSchema()
#df_credit2.show(5)


In [22]:
#changing datatype of columns

#df_credit2= df_credit2.withColumn("TIMEID",col("TIMEID").cast("String"))
#df_credit2 = df_credit2.withColumn("CUST_SSN",col("CUST_SSN").cast("int"))
#df_credit2= df_credit2.withColumn("TRANSACTION_ID",col("TRANSACTION_ID").cast("int"))
#df_credit2 = df_credit2.withColumn("BRANCH_CODE",col("BRANCH_CODE").cast("int"))
#df_credit2.show(3)
#df_credit2.printSchema()

# Function Requirement 1.2

Data loading into Database

In [24]:
credit_df.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone_") \
.option("dbtable", "creditcard_capstone_.CDW_SAPP_CREDIT_CARD") \
.option("user", "root") \
.option("password", "a") \
.save()